In [ ]:
[Reference](https://www.tensorflow.org/overview/)

In [2]:
import tensorflow as tf
import numpy
tf.enable_eager_execution()

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [5]:

tf.executing_eagerly()

True

In [6]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.78185475,  0.24619973,  0.14707181, -0.53637624, -0.05262943,
         0.15731092, -0.35579374, -0.09316206,  0.8643328 , -0.20718268]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.04400827, 0.12303045, 0.11141969, 0.05625281, 0.09124999,
        0.11256638, 0.06738608, 0.08762533, 0.22827819, 0.07818279]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [9]:
loss_fn(y_train[:1], predictions).numpy()

2.1842122

In [10]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [11]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 2s 35us/sample - loss: 0.3032 - acc: 0.9123
Epoch 2/5
60000/60000 [==============================] - 2s 31us/sample - loss: 0.1441 - acc: 0.9573
Epoch 3/5
60000/60000 [==============================] - 2s 31us/sample - loss: 0.1065 - acc: 0.9684
Epoch 4/5
60000/60000 [==============================] - 2s 31us/sample - loss: 0.0864 - acc: 0.9738
Epoch 5/5
60000/60000 [==============================] - 2s 31us/sample - loss: 0.0744 - acc: 0.9763


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [12]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 0s - loss: 0.0704 - acc: 0.9796


[0.07042095393759665, 0.9796]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [13]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [14]:
probability_model(x_test[:5])

<tf.Tensor: id=68331, shape=(5, 10), dtype=float32, numpy=
array([[3.33282166e-08, 1.28785985e-07, 1.69062127e-06, 6.12572912e-05,
        6.75208639e-11, 2.71155187e-07, 6.55768164e-14, 9.99925971e-01,
        1.68313022e-07, 1.05954659e-05],
       [1.60721504e-06, 3.36002966e-04, 9.99661088e-01, 1.32284731e-06,
        5.46347898e-16, 2.73854166e-08, 3.21673599e-09, 2.08637881e-12,
        2.22717897e-08, 1.62685027e-14],
       [1.77266998e-07, 9.99493480e-01, 2.05043689e-05, 7.73060765e-06,
        8.39552195e-06, 4.61082027e-06, 1.34734410e-06, 3.66496970e-04,
        9.67942397e-05, 3.83179469e-07],
       [9.99971628e-01, 1.35657885e-08, 1.85506360e-06, 3.31388605e-09,
        9.42317158e-08, 6.19589429e-08, 4.62314210e-06, 2.11739698e-05,
        1.01226059e-08, 5.75155525e-07],
       [3.92618040e-06, 9.69101777e-09, 6.65038078e-06, 2.81810824e-08,
        9.96943414e-01, 5.06634888e-06, 3.51616450e-06, 5.76817620e-05,
        5.32535807e-07, 2.97911302e-03]], dtype=float32)>